In [23]:
import requests
import os
from configparser import ConfigParser

#Open https://app.bytenite.com/profile in your browser and copy the access token in the variable below
parser = ConfigParser()
_ = parser.read('api.cfg')
base_url = 'https://api.bytenite.com'
storj_access_key=parser.get('api', 'storj_access_key')
storj_secret_key=parser.get('api', 'storj_secret_key')
# Get user info
resp = requests.get(f'{base_url}/v1/auth/user', headers={'Authorization': parser.get('api', 'access_token')})
resp.raise_for_status()
user = resp.json()['user']
print(user)

{'id': '7eJ03LAvszYNRasmgBxMPqrEM6r1', 'createdAt': '2023-01-04T16:35:26.928Z', 'lastLogin': '2023-05-15T16:43:28.230Z', 'displayName': 'n.castelli', 'email': 'n.castelli@bytenite.com', 'phoneNumber': '', 'photoUrl': '', 'providerId': 'firebase', 'emailVerified': True, 'customClaims': {}}


In [24]:
# Set a data source and a data destination
data_source_body={
    "dataSource": {
        "dataSourceDescriptor":"storj",
        "params":{
          "@type": "type.googleapis.com/bytenite.data_source.StorjDataSource",
          "accessKey": storj_access_key,
          "cloudRegion": "us1",
          "secretKey": storj_secret_key,
          "bucketName": "input-bytenite",
          "name": "vqc/A086.mp4",
        }
    },
    "dataDestination": {
        "dataSourceDescriptor":"storj",
        "params":{
          "@type": "type.googleapis.com/bytenite.data_source.StorjDataSource",
          "accessKey": storj_access_key,
          "bucketName": "output-bytenite",
          "cloudRegion": "us1",
          "name": "Test api A086",
          "secretKey": storj_secret_key
        }
    }
}


In [25]:
# Set video encoding parameters
job_params_body={
    "data": {
      "output_code_template": "",
      "output_template": "{{.name}}",
      "outputs": [
        {
          "output_params": {
            "aspect": {
              "cropping": {
                "bottom": 0,
                "left": 0,
                "right": 0,
                "top": 0
              },
              "orientation": {
                "flip": "",
                "rotation": ""
              },
              "padding": {
                "fill_color": "#000000",
                "final_aspect_ratio": ""
              },
              "resolution": {
                "aspect_ratio": "",
                "avoid_upscaling": False,
                "height": "720p"
              }
            },
            "audio": {
              "audio_bitrate": "320k",
              "audio_channel": "2.0",
              "audio_codec": "AAC",
              "audio_sample_rate": "48khz"
            },
            "video": {
              "bitrate": {
                "avg_bitrate": "3000k",
                "buffer_size": "8000k",
                "max_bitrate": "8000k",
                "min_bitrate": "700k",
                "rate_control_mode": "variable_bitrate"
              },
              "codec": "libx264",
              "codec_params": {
                "level": "",
                "preset": "slow",
                "profile": "",
                "tune": "film"
              },
              "frame_rate": {
                "fps": "",
                "up_mode": "avoid"
              }
            }
          },
          "output_type": "mp4",
          "name": "720p"
        },
        {
          "output_params": {
            "aspect": {
              "cropping": {
                "bottom": 0,
                "left": 0,
                "right": 0,
                "top": 0
              },
              "orientation": {
                "flip": "",
                "rotation": ""
              },
              "padding": {
                "fill_color": "#000000",
                "final_aspect_ratio": ""
              },
              "resolution": {
                "aspect_ratio": "",
                "avoid_upscaling": False,
                "height": "480p"
              }
            },
            "audio": {
              "audio_bitrate": "320k",
              "audio_channel": "2.0",
              "audio_codec": "AAC",
              "audio_sample_rate": "48khz"
            },
            "video": {
              "bitrate": {
                "avg_bitrate": "800k",
                "buffer_size": "2000k",
                "max_bitrate": "2000k",
                "min_bitrate": "400k",
                "rate_control_mode": "variable_bitrate"
              },
              "codec": "libvpx-vp9",
              "codec_params": {
                "cpu_used": 1
              },
              "frame_rate": {
                "fps": "",
                "up_mode": "avoid"
              }
            }
          },
          "output_type": "mp4",
          "name": "480p"
        }
      ]
    },
    "preset": "unpackaged"
}

In [30]:
# Create job with data source and params
create_job_body={"name": "Example job", "templateId": "video-transcoding@v0.2", "params": job_params_body, "data_source": data_source_body["dataSource"], "data_destination": data_source_body["dataDestination"]}

job_data = requests.post(f'{base_url}/v1/customer/jobs', json=create_job_body, headers={'Authorization': parser.get('api', 'access_token')}).json()['job']
job_id = job_data['id']
print(job_data['name'])

Example job


In [27]:
# Get time and cost estimation
#
resp = requests.get(f'{base_url}/v1/customer/jobs/estimation/{job_id}', headers={'Authorization': parser.get('api', 'access_token')})
resp.raise_for_status()
estimation_data = resp.json()
total_estimation = estimation_data['estimation']['totalEstimation']
print(f'Cost estimation: {total_estimation} ByteChips = {(int(total_estimation)/200):.02f}$')

Cost estimation: 2 ByteChips = 0.01$


In [28]:
# Run the job
run_resp = requests.post(f'{base_url}/v1/customer/jobs/run/{job_id}', json={}, headers={'Authorization': parser.get('api', 'access_token')}).json()
print(run_resp)
print(f'Job {job_id} started')

{'ok': True, 'requestId': '', 'message': '', 'error': None}
Job e7e86b18-92de-49ae-beb1-32a17a351685 started
